In [7]:
# Dependencies and Setup
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy.stats import linregress
import scipy.stats as st

# Data files
zips_ks_path = "resources/ks_zipcodes.csv"
zips_mo_path = "resources/mo_zipcodes.csv"

# Read the data
zips_ks = pd.read_csv(zips_ks_path)
zips_mo = pd.read_csv(zips_mo_path)

# Combine the data into a single datasets
zips_df = pd.concat([zips_ks, zips_mo])
zips_df

,Zip Code,Zip Code Type,Zip Code Name,County
0,66002,Non-Unique,Atchison,Atchison
1,66006,Non-Unique,Baldwin City,Douglas
2,66007,Non-Unique,Basehor,Leavenworth
3,66008,Non-Unique,Bendena,Doniphan
4,66010,Non-Unique,Blue Mound,Linn
...,...,...,...,...
1153,65817,PO Box,Springfield,Greene
1154,65890,Unique,Springfield,Greene
1155,65897,Unique,Springfield,Greene
1156,65898,Unique,Springfield,Greene
